# Bài 2

In [1]:
# Import các thư viện cần thiết
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Đọc nội dung file NetworkConf.xml
with open('../Lab3/data/NetworkConf.xml','r', encoding = 'utf-8') as f:
    NetworkConf = f.read()

# Đọc nội dung file TermConf.xml
with open('../Lab3/data/TermConf.xml','r',encoding='utf-8') as f:
    TermConf = f.read()

# Parse nội dung file NetworkConf.xml bằng BeautifulSoup
NetworkConf = BeautifulSoup(NetworkConf,'xml')
NetworkConf = NetworkConf.find_all('NetworkSession')

# Parse nội dung file TermConf.xml bằng BeautifulSoup
TermConf = BeautifulSoup(TermConf,'xml')
TermConf = TermConf.find('Terminal')

### 1. Hiển thị thông tin theo yêu cầu của đề bài

#### Trích xuất dữ liệu

In [3]:
def get_data_from_NetworkConf(soup: BeautifulSoup) -> dict:
    '''Lấy các dữ liệu theo yêu cầu của đề bài trong file NetworkConf.xml'''
    groupProtpcol = {}

    for _ in soup:
        # Lấy thuộc tính `name` và `protocol` từ tag NetworkSession
        name = _.get('name').strip()
        protocol = _.get('protocol').strip()

        # Lấy thông tin địa chỉ mạng từ tag NetworkAddress
        NetworkAddress = _.find('NetworkAddress')
        Ip, Port = NetworkAddress.get('hostIP').strip().split(':')

        # Chuyển đổi giá trị `enable` thành trạng thái kết nối
        dictEnable = {"0": "không kết nối", "1": "đang kết nối"}
        enable = _.get('enable').strip()
        status = dictEnable.get(enable).strip()

        # Nhóm các session theo protocol
        if protocol not in groupProtpcol:
            groupProtpcol[protocol] = []
        groupProtpcol[protocol].append((name, Ip, Port, enable, status))

    return groupProtpcol

In [4]:
def get_data_from_TermConf(soup: BeautifulSoup) -> list:
    '''Lấy các dữ liệu theo yêu cầu của đề bài trong file TermConf.xml'''
    lst_termConf = []

    # Lấy thuộc tính `id` và các thông tin con từ tag Terminal
    id = soup.get('id').strip()
    TermIP = soup.find('TermIP').text.strip()
    branchNumber = soup.find('branchNumber').text.strip()
    TerminalType = soup.find('TerminalType').text.strip()

    # Thêm thông tin vào danh sách
    lst_termConf.append([id, TermIP, branchNumber, TerminalType]) 

    return lst_termConf

#### Hiển thị dữ liệu

In [5]:
# Hiển thị dữ liệu từ file NetworkConf.xml
print(f"Hiện thị thông tin theo từng Protocol")
for _ in get_data_from_NetworkConf(NetworkConf):
    print(f"\nProtocol: {_}")

    for name, Ip, Port, enable, status in get_data_from_NetworkConf(NetworkConf)[_]:
        print(f"\tname: {name}, IP: {Ip}, Port: {Port}, enable:{enable}, status: {status}")
        
print('---'*25)

# Hiển thị dữ liệu từ file TermConf.xml
print(f"Thông tin của ATM\n")
for id, TermIP, branchNumber, TerminalType in get_data_from_TermConf(TermConf):
    print(f"ID: {id}\nIP: {TermIP}\nBranch Number: {branchNumber}\nTerminal Type: {TerminalType}")

Hiện thị thông tin theo từng Protocol

Protocol: ISO8583
	name: ATMP1, IP: 10.1.8.11, Port: 1234, enable:0, status: không kết nối
	name: ATMP3, IP: 21.96.3.137, Port: 9112, enable:0, status: không kết nối

Protocol: NDC/DDC
	name: ATMP2, IP: 10.30.1.65, Port: 8081, enable:1, status: đang kết nối

Protocol: xml/json
	name: RESTFUL1, IP: 10.1.24.86, Port: 8089, enable:0, status: không kết nối
	name: RESTFUL2, IP: 10.1.25.88, Port: 8088, enable:1, status: đang kết nối
---------------------------------------------------------------------------
Thông tin của ATM

ID: 27082005
IP: 192.168.1.9
Branch Number: 2005
Terminal Type: abcdef


---

### 2. 

In [6]:
import xml.etree.ElementTree as ET
import socket

# Load XML
term_tree = ET.parse("../Lab3/data/TermConf.xml")
term_root = term_tree.getroot()

network_tree = ET.parse("../Lab3/data/NetworkConf.xml")
network_root = network_tree.getroot()


In [7]:
def get_local_ip():
    '''Lấy địa chỉ IP cục bộ của máy'''
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

In [8]:
def update_terminal_info():
    '''Cập nhật thông tin Terminal trong file TermConf.xml'''
    ip = get_local_ip()  # Lấy IP cục bộ
    terminal = term_root.find("Terminal")

    # Cập nhật các thông tin Terminal
    terminal.find("TermIP").text = ip
    terminal.set("id", input("Nhập Terminal ID mới: "))
    terminal.find("branchNumber").text = input("Nhập Branch Number: ")
    terminal.find("TerminalType").text = input("Nhập Terminal Type: ")
    terminal.find("autorestartmode").text = input("Nhập Auto Restart Mode (0 hoặc 1): ")

    # Ghi lại file TermConf.xml
    term_tree.write("../Lab3/data/TermConf.xml", encoding="utf-8", xml_declaration=True)
    print("Đã cập nhật TerminalConf.xml")

In [9]:
def update_network_session():
    '''Cập nhật thông tin session trong file NetworkConf.xml'''
    sessions = network_root.find(".//Sessions")
    all_sessions = sessions.findall("NetworkSession")
    
    # Hiển thị danh sách các session
    print("\nDanh sách sessions:")
    for idx, s in enumerate(all_sessions):
        print(f"{idx+1}. {s.attrib.get('name')} - Protocol: {s.attrib.get('protocol')}")

    # Chọn session để cập nhật
    choice = int(input("Chọn session muốn cập nhật (nhập số): ")) - 1
    session = all_sessions[choice]

    # Cập nhật thông tin session
    new_ip = input("Nhập IP:Port mới cho session: ")
    session.find("NetworkAddress").attrib["hostIP"] = new_ip
    session.set("name", input("Nhập name mới: "))
    session.set("desc", input("Nhập mô tả (desc): "))
    session.set("protocol", input("Nhập protocol mới: "))

    # Nếu protocol là NDC/DDC, cập nhật thêm thông tin bảo mật
    if session.attrib["protocol"] == "NDC/DDC":
        sec = session.find(".//Security/Auth")
        if sec is not None:
            sec.set("serverX509CertificateName", input("Nhập serverX509CertificateName mới: "))
            sec.set("sslProtocol", input("Nhập sslProtocol mới (VD: Tls12): "))

    # Ghi lại file NetworkConf.xml
    network_tree.write("../Lab3/data/NetworkConf.xml", encoding="utf-8", xml_declaration=True)
    print("Đã cập nhật NetworkConf.xml")

In [10]:
def main():
    '''Hàm chính để hiển thị menu và xử lý lựa chọn'''
    while True:
        print("\n========== MENU ==========")
        print("1. Cập nhật TerminalConf (IP, ID, Branch, Type, AutoRestart)")
        print("2. Cập nhật NetworkConf (session info)")
        print("0. Thoát")
        choice = input("Chọn chức năng: ")

        if choice == "1":
            update_terminal_info()  # Gọi hàm cập nhật TerminalConf
        elif choice == "2":
            update_network_session()  # Gọi hàm cập nhật NetworkConf
        elif choice == "0":
            break  # Thoát chương trình
        else:
            print("Lựa chọn không hợp lệ!")

main()


========== MENU ==========
1. Cập nhật TerminalConf (IP, ID, Branch, Type, AutoRestart)
2. Cập nhật NetworkConf (session info)
0. Thoát

Danh sách sessions:
1. ATMP1 - Protocol: ISO8583
2. ATMP2 - Protocol: NDC/DDC
3. RESTFUL1 - Protocol: xml/json
4. RESTFUL2 - Protocol: xml/json
5. ATMP3 - Protocol: ISO8583
Đã cập nhật NetworkConf.xml

========== MENU ==========
1. Cập nhật TerminalConf (IP, ID, Branch, Type, AutoRestart)
2. Cập nhật NetworkConf (session info)
0. Thoát
